<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Daniel Carmona G.
- Nombre de alumno 2: Consuelo Rojas N.


### **Link de repositorio de GitHub:** `https://github.com/DanielCG19/Evaluaciones-MDS7202`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [2]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

!pip install xgboost
from xgboost import XGBClassifier
# import lightgbm as lgb

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 7.0 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 128.2 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.10.0
    Not uninstalling plotly at /shared-libs/python3.9/py/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'plotly'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 141.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 124.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 

True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [3]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [4]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']).drop(columns=['Unnamed: 0'])# eliminar ejemplos sin historia

In [5]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,55,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [6]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [nltk.word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [7]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [8]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [9]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [10]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [11]:
#### Código aquí ####
ct = ColumnTransformer(
    [('CountVectorizer', CountVectorizer(tokenizer= StemmerTokenizer(),ngram_range=(1,2)), 'history_text'),
    ("MinMax", MinMaxScaler(), ['intelligence_score', 'strength_score','speed_score', 'durability_score', 'power_score', 'combat_score'])
       ]
    )

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [12]:
#### Código aquí ####
steps = [("Preprocesamiento", ct),
         ("FeatureSelector",SelectPercentile(f_classif, percentile=90)),
         ('Clf', MultinomialNB())]

stepsDummy = [("Preprocesamiento", ct),
         ("FeatureSelector",SelectPercentile(f_classif, percentile=90)),
         ('Clf', DummyClassifier(strategy="stratified"))] 

pipe = Pipeline(steps)
pipeDummy = Pipeline(stepsDummy)

In [13]:
X = df_comics.drop(['alignment'], axis=1).reset_index(drop=True) # probar con one hot par nacionalidad
y = df_comics['alignment']#.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [14]:
X_train.shape #.iloc[0,:].shape

(899, 80)

In [15]:
y_train = pd.DataFrame(y_train.reset_index(drop=True).to_numpy())#.shape

In [16]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = pd.DataFrame(le.transform(y_train))
y_train.head()

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,0
0,0
1,1
2,2
3,0
4,1


In [17]:
y_test = pd.DataFrame(le.transform(y_test))
y_test.head()

,0
0,0
1,0
2,2
3,1
4,0


In [18]:
y_train.shape

(899, 1)

In [19]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

pipeDummy.fit(X_train, y_train)
y_predDummy = pipeDummy.predict(X_test)

print(f"NB:\n {classification_report(y_test, y_pred)}")
print(f"Dummy:\n {classification_report(y_test, y_predDummy)}")

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
NB:
               precision    recall  f1-score   support

           0       0.67      0.06      0.11       129
           1       0.59      0.99      0.74       223
           2       0.00      0.00      0.00        34

    accuracy                           0.59       386
   macro avg       0.42      0.35      0.28       386
weighted avg       0.56      0.59      0.47       386

Dummy:
               precision    recall  f1-score   suppor

In [22]:
le.inverse_transform([0,1,2])
#y_test.value_counts()

array(['Bad', 'Good', 'Neutral'], dtype=object)

```
Escriba aquí su justificación   

Como podemos notar de los resultados entregados por classification report, podemos notar que el accuracy del modelo Dummy, el cual no considera los valores de los atributos de entrada para la predicción, es de un 49%. Esto significa que aproximadamente, el 50% de las veces el modelo acerta a la categoría real del personaje. 

Si se tiene en cuenta que hay 3 etiquedas distintas: Héroe, Villano y Neutral, se puede decir que un DummyModel es mejor que una elección al azar, dado que esta debería dar un porcetaje de acierto, cercano al 33,3%.

Por otro lado, se tiene que con el clasificador Naive Bayes hay un mejor accuracy y mejor precisión generalizada. Sin embargo, tiene muy malos parametros de recall, esto significa que el real porcentaje de etiquetado de las etiquetas 0 y 2 es muy bajo y solo logra etiquetar correctamente a la clase 1, la cual corresponde a Good o Héroe.

Esto también se ve en el modelo dummy, el cual tiene porcentajes de precision menores para las clases 0 y 2, ya que el conjunto de datos se encuentra desbalanceado, teniendo una mayor cantidad de datos de la clase Good, seguido de la clase Bad y por ultimo muy pocos datos de la clase Neutral. Proporciones que coinciden con los porcentajes de precision con que se determinaron estas clases.
```

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 4:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [37]:
pipe.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('Preprocesamiento',
                 ColumnTransformer(transformers=[('CountVectorizer',
                                                  CountVectorizer(ngram_range=(1,
                                                                               2),
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7f28076dce20>),
                                                  'history_text'),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score'])])),
 

In [62]:
# params = [
#        # clasificador 1 + hiperparámetros
#        {'Preprocesamiento__CountVectorizer__ngram_range': [(1,1),(1,2),(1,3)],
#         'FeatureSelector__percentile': [40, 60, 80, 90],
#         'Clf': [MultinomialNB()]} ,

#     #    # clasificador 2 + hiperparámetros
#        {'Preprocesamiento__CountVectorizer__ngram_range': [(1,1),(1,2),(1,3)],
#         'FeatureSelector__percentile': [40, 60, 80, 90],
#         'Clf': [RandomForestClassifier()]}  ,

#     #    # clasificador 3 + hiperparámetros
#        {'Preprocesamiento__CountVectorizer__ngram_range': [(1,1),(1,2),(1,3)],
#         'FeatureSelector__percentile': [40, 60, 80, 90],
#         'Clf':  [XGBClassifier(objective='binary:logistic',
#                           booster='gbtree',
#                           eval_metric='auc',
#                           tree_method='hist',
#                           grow_policy='lossguide')]}  
#        ]
params = {'Preprocesamiento__CountVectorizer__ngram_range': [(1,1),(1,2),(1,3)],
          'FeatureSelector__percentile': [40, 60, 80, 90],
          'Clf': [MultinomialNB(), RandomForestClassifier(), XGBClassifier()]}

search = HalvingGridSearchCV(pipe, params,
                             error_score='raise',
                             verbose=10,
                             random_state=0).fit(X_train, y_train)
search.best_params_ 

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 33
max_resources_: 899
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 36
n_resources: 33
Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5; 1/36] START Clf=MultinomialNB(), FeatureSelector__percentile=40, Preprocesamiento__CountVectorizer__ngram_range=(1, 1)
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[CV 1/5; 1/36] END Clf=MultinomialNB(), FeatureSelector__pe

KeyboardInterrupt: 

In [64]:
best_params = {'Clf': MultinomialNB(),
 'FeatureSelector__percentile': 80,
 'Preprocesamiento__CountVectorizer__ngram_range': (1, 1)}

In [66]:
ct = ColumnTransformer(
    [('CountVectorizer', CountVectorizer(tokenizer= StemmerTokenizer(),ngram_range=(1,1)), 'history_text'),
    ("MinMax", MinMaxScaler(), ['intelligence_score', 'strength_score','speed_score', 'durability_score', 'power_score', 'combat_score'])
       ]
    )
steps = [("Preprocesamiento", ct),
         ("FeatureSelector",SelectPercentile(f_classif, percentile=80)),
         ('Clf', MultinomialNB())]
pipe = Pipeline(steps)

In [51]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
              precision    recall  f1-score   support

         Bad       0.61      0.54      0.57       129
        Good       0.69      0.83      0.75       223
     Neutral       0.40      0.06      0.10        34

    accuracy                           0.66       386
   macro avg       0.57      0.48      0.48       386
weighted avg       0.64      0.66      0.64       386

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


```
Los resultados de la grilla de búsqueda dieron que el mejor modelo, es el Multinomial Naive Bayes, el mismo clasificador que el baseline. Sin embargo, si se ven las métricas, de manera general, la certeza del modelo mejoro en 0.06 puntos, lo que se traduce en mejoras globales en la precisión del modelo. Además, hay que notar que para todas las etiquetas, el recall aumentó, lo que significa que el porcentaje de aciertos también aúmento.

Esto se debe a la selección de hiperparámetros. La búsqueda de hiperparámetros se hace para poder generar un modelo confiable y apto para el problema a solucionar. Además, como no es posible obtener estos desde los datos, es necesario hacer una exploración de estos para obtener la mejor combinación posible, que permita un buen desempeño.

Sobre los otros modelos probados, se utilizó un RandomForestClassifier() y un XGBClassifier(). Se sabe que un RandomForest funciona de manera similar a los árboles de decisión, por lo que estos, a pesar de solucionar en gran parte el problema del sesgo que producen, siguen siendo propensos a sesgarse cuando los datos de entrenamiento no estan bien balanceados. Por ello, se entiende que la útilización de este clasificador no es la más adecuada para el problema.

Por otro lado, se tiene el XGBClassifier, este clasificador se supone que es un boost de árboles de decisión, por lo que da a entender que podría, eventualmente, tener problemas con datos no bien balanceados, razon por la cual el desempeño de este clasificador no fue mejor que el de naive bayes al momento de buscar los mejores hiperparámetros.

Cabe destacar que esto podría cambiar si se varian el numero de hiperparámetros a probar con cada modelo, o si se añaden hiperparámetros especificos de cada modelo que permitan un mejor desempeño en el dataset utilizado.


```

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [60]:
#### Código aquí ####
#X_test[X_test["name"]=="Vergil"]
df_nl = df_comics_no_label.dropna(subset=['history_text']).drop(columns=['Unnamed: 0'])# eliminar ejemplos sin historia

vergil = df_nl[df_nl["name"]=="Vergil"]
gorillaGirl = df_nl[df_nl["name"]=="Gorilla Girl"]
Batcow = df_nl[df_nl["name"]=="Batcow"]


In [61]:
Batcow

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
16,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
vergil_pred = le.inverse_transform(pipe.predict(vergil))
gorillaGirl_pred = le.inverse_transform(pipe.predict(gorillaGirl))
Batcow_pred = le.inverse_transform(pipe.predict(Batcow))

print(f"Vergil: {vergil_pred}\n Gorilla Girl: {gorillaGirl_pred}\n Batcow: {Batcow_pred}")

Vergil: ['Neutral']
 Gorilla Girl: ['Good']
 Batcow: ['Good' 'Good']


De los resultados obtenidos, se verificó que tanto Gorrila Girl como Batcow son personajes considerados buenos, por lo que el clasificador estaría en lo correcto para estas dos predicciones. En cuanto al personaje de Vergil, no sabemos si este realmente se trataba de un personaje neutral o villano.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9fe4af16-9ce4-4502-87cb-c2aa33982b6b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>